# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import wandb

In [ ]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, HistGradientBoostingRegressor, ExtraTreesRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor

# Import Preprocessed Data

In [ ]:
data = pd.read_csv("/content/Processed_dataset.csv", header=0, sep=',', encoding='Latin1')

# Label Encoding

In [ ]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']

mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(data[i])
    tr = mapper[i].transform(data[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    data.loc[:, i+'_labels'] = pd.Series(tr, index=data.index)
    
labeled = data[['price', 'yearOfRegistration','powerPS','kilometer','monthOfRegistration']
                  +[x+"_labels" for x in labels]]

print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


# Score Evaluation

In [ ]:
def find_scores(Y_actual, Y_pred, X_train):
    mae = mean_absolute_error(Y_actual, Y_pred)
    mse = mean_squared_error(Y_actual, Y_pred)
    rmse = np.sqrt(mse)
    rmsle = np.log(rmse)
    r2 = r2_score(Y_actual, Y_pred)
    n, k = X_train.shape
    adj_r2_score = 1 - ((1-r2)*(n-1)/(n-k-1))
    
    wandb.log({"mae": mae, "mse": mse, 'rmse':rmse, 'rmsle':rmsle, 'r2':r2, 'adj_r2':adj_r2_score})

# Predictive Modeling

# Bagging Regressor

In [ ]:
def bagging_regressor():
    config_defaults = {
                'n_estimators':100,
                'max_samples':0.4,
                'bootstrap':True,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = BaggingRegressor(
      n_estimators=config.n_estimators, 
      bootstrap=config.bootstrap,
      max_samples=config.max_samples,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [ ]:
bagging_regressor_configs = {
    "name":'BaggingRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "n_estimators": {
            "values": [100, 200, 300]
        },
        "max_samples": {
            "values": [0.4,0.5, 0.6]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep=bagging_regressor_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=bagging_regressor)

Create sweep with ID: qazs1wvp
Sweep URL: https://wandb.ai/rajeshtr/car_resale_value/sweeps/qazs1wvp


wandb: Agent Starting Run: 4r3ljwa1 with config:
wandb: 	max_samples: 0.4
wandb: 	n_estimators: 100
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


C:\Users\Rajesh\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:429: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.85951
mae,1360.56321
mse,10013619.88798
r2,0.85952
rmse,3164.43042


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ygrrap15 with config:
wandb: 	max_samples: 0.4
wandb: 	n_estimators: 200
wandb: Currently logged in as: rajeshtr. Use `wandb login --relogin` to force relogin


...

adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86257
mae,1347.26055
mse,9795571.47319
r2,0.86258
rmse,3129.78777


wandb: Agent Starting Run: b3z35ga8 with config:
wandb: 	max_samples: 0.6
wandb: 	n_estimators: 300


C:\Users\Rajesh\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:429: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86263
mae,1346.42732
mse,9791114.16887
r2,0.86264
rmse,3129.07561


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# Bagging Regressor Results

![W&B%20Chart%2011_10_2022,%201_44_32%20PM.png](attachment:W&B%20Chart%2011_10_2022,%201_44_32%20PM.png)

# RandomForest Regressor

In [ ]:
def random_forest_regressor():
    config_defaults = {
                'n_estimators':100,
                'max_samples':0.4,
                'criterion':'squared_error',
                'bootstrap': True,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = RandomForestRegressor(
      n_estimators=config.n_estimators,
      criterion = config.criterion,
      bootstrap=config.bootstrap,
      max_samples=config.max_samples,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [ ]:
random_forest_configs = {
    "name":'RandomForestRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "n_estimators": {
            "values": [100, 200, 300]
        },
        "max_samples": {
            "values": [0.4,0.5, 0.6]
        }
    }
}

sweep_id = wandb.sweep(sweep=random_forest_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=random_forest_regressor)

Create sweep with ID: xvbhv8cq
Sweep URL: https://wandb.ai/rajeshtr/car_resale_value/sweeps/xvbhv8cq


wandb: Agent Starting Run: d9m5p5al with config:
wandb: 	max_samples: 0.4
wandb: 	n_estimators: 100


C:\Users\Rajesh\AppData\Local\Temp\ipykernel_33400\3309427692.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, Y_train)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86019
mae,1359.65132
mse,9965260.7893
r2,0.8602
rmse,3156.78013


wandb: Agent Starting Run: r4qy4pa3 with config:
wandb: 	max_samples: 0.4
wandb: 	n_estimators: 200


...

adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86265
mae,1347.34878
mse,9789829.20211
r2,0.86266
rmse,3128.87028


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: bccpklwb with config:
wandb: 	max_samples: 0.6
wandb: 	n_estimators: 300


C:\Users\Rajesh\AppData\Local\Temp\ipykernel_33400\3309427692.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, Y_train)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86281
mae,1346.18468
mse,9778805.08474
r2,0.86281
rmse,3127.1081


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# Random Forest Regressor Results

![W&B%20Chart%2011_10_2022,%202_05_41%20PM.png](attachment:W&B%20Chart%2011_10_2022,%202_05_41%20PM.png)

# LGBM Regressor

In [ ]:
def LGBM_regressor():
    config_defaults = {
                'objective':'root_mean_squared_error',
                'reg_sqrt': True,
                'metric':'rmse',
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = LGBMRegressor(
      learning_rate=config.learning_rate,
      n_estimators = config.n_estimators,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [ ]:
lgbm_configs = {
    "name":'LGBMRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "values": [0.01, 0.03, 0.05, 0.07]
        },
        "objective": {
            "values": ['root_mean_squared_error']
        },
        "boosting_type": {
            "values": ['gbdt','dart','goss','rf']
        },
        "reg_sqrt": {
            "values": [True]
        },
        "metric": {
            "values": ['rmse']
        },
        "n_estimators": {
            "values": [100,200,300]
        },
        "random_state": {
            "values": [42]
        }
    }
}

sweep_id = wandb.sweep(sweep=lgbm_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=LGBM_regressor)

Create sweep with ID: b30glh4n
Sweep URL: https://wandb.ai/rajeshtr/car_resale_value/sweeps/b30glh4n


wandb: Agent Starting Run: 38h98sq1 with config:
wandb: 	boosting_type: gbdt
wandb: 	learning_rate: 0.01
wandb: 	metric: rmse
wandb: 	n_estimators: 100
wandb: 	objective: root_mean_squared_error
wandb: 	random_state: 42
wandb: 	reg_sqrt: True


C:\Users\Rajesh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.63522
mae,2868.23264
mse,26000513.79616
r2,0.63524
rmse,5099.0699


wandb: Agent Starting Run: 1ybay1vn with config:
wandb: 	boosting_type: gbdt
wandb: 	learning_rate: 0.01
wandb: 	metric: rmse
wandb: 	n_estimators: 200
wandb: 	objective: root_mean_squared_error
wandb: 	random_state: 42
wandb: 	reg_sqrt: True


...

adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86505
mae,1400.49904
mse,9618944.77269
r2,0.86506
rmse,3101.44237


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 3qrld772 with config:
wandb: 	boosting_type: rf
wandb: 	learning_rate: 0.07
wandb: 	metric: rmse
wandb: 	n_estimators: 300
wandb: 	objective: root_mean_squared_error
wandb: 	random_state: 42
wandb: 	reg_sqrt: True


C:\Users\Rajesh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86949
mae,1366.21695
mse,9302686.85872
r2,0.86949
rmse,3050.03063


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# LGBM Regressor Results

![W&B%20Chart%2011_10_2022,%203_20_48%20PM.png](attachment:W&B%20Chart%2011_10_2022,%203_20_48%20PM.png)